In [1]:
from gensim.parsing.preprocessing import *

In [12]:
FILTERS = [
    stem_text,
    strip_tags,
    strip_punctuation,
    strip_multiple_whitespaces,
    remove_stopwords,
]

In [15]:
import pandas as pd

post_df = pd.read_json("data/preprocessed_data.jsonl", orient="records")

In [31]:
flattened = []
for tags in post_df.head().tags.to_list():
    flattened += tags
flattened

['jolyne cujoh',
 'giorno giovanna',
 'josuke higashikata',
 'kitten sized',
 'myth',
 'fengqing',
 'mu qing',
 'feng xin',
 '🥰 just a little boy 🚯 i will kiss 💋 your heart',
 'so true']

In [32]:
from collections import OrderedDict

canonical = OrderedDict()
for word in flattened:
    p_word = ' '.join(preprocess_string(word, FILTERS))
    if p_word not in canonical:
        canonical[p_word] = [word]
    else:
        canonical[p_word].append(word)
canonical

OrderedDict([('jolyn cujoh', ['jolyne cujoh']),
             ('giorno giovanna', ['giorno giovanna']),
             ('josuk higashikata', ['josuke higashikata']),
             ('kitten size', ['kitten sized']),
             ('myth', ['myth']),
             ('fengq', ['fengqing']),
             ('mu qing', ['mu qing']),
             ('feng xin', ['feng xin']),
             ('🥰 littl boi 🚯 kiss 💋 heart',
              ['🥰 just a little boy 🚯 i will kiss 💋 your heart']),
             ('true', ['so true'])])

In [41]:
import numpy as np
for x in np.mean(np.array([[1,2,3],[2,1,3]]), axis=0):
    print(x)

1.5
1.5
3.0


In [27]:
from transformers import AutoTokenizer, BertConfig, BertForMaskedLM

from txtai.pipeline import HFTrainer

config = BertConfig(
    vocab_size = 500,
    hidden_size = 50,
    num_hidden_layers = 2,
    num_attention_heads = 2,
    intermediate_size = 100,
)

model = BertForMaskedLM(config)
model.save_pretrained("bert")
tokenizer = AutoTokenizer.from_pretrained("bert")

train = HFTrainer()

# Train model
train((model, tokenizer), dataset, task="language-modeling", output_dir="bert",
      fp16=True, per_device_train_batch_size=128, num_train_epochs=10,
      dataloader_num_workers=2)

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.